In [1]:
from model.modelorchastrator import ModelOrchestrator
import pandas as pd
from data_pipeline.dataset import Dataset
from data_pipeline.pipelinesteps import data_splitter
from hydra.utils import instantiate
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from functools import partial
from data_pipeline.pipelinesteps import load_data
from pathlib import Path
from sklearn.ensemble import HistGradientBoostingClassifier
from model.feature_selector import FeatureSelector
from model.modelorchastrator import OptBinningTransformer
import os

(CVXPY) Mar 26 03:45:45 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Mar 26 03:45:45 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
 2024-03-26 15:45:45,983 -  INFO  	: my_logger.custom_logger - Logger is set up.


In [2]:

GlobalHydra.instance().clear()
initialize(config_path="../", version_base=None)
cfg = compose(config_name="config")

In [3]:

base_path = Path(os.getcwd()).parent
dataset = Dataset.create_from_pipeline(
                partial(load_data, base_path), 
                instantiate(cfg.data_pipeline),
                data_splitter=data_splitter,
                target_column="target",
        )

In [4]:
model_orchestrator = ModelOrchestrator(cfg)

In [5]:
pipe = model_orchestrator.modelpipeline

In [6]:
# pipe.fit(dataset.X_train, dataset.y_train)

In [17]:
from optuna import create_study
from optuna import Trial
from  optuna import samplers

from sklearn.metrics import roc_auc_score


def objective(trial : Trial) -> float:
  model_orchestrator = ModelOrchestrator(cfg, trial)
  pipe = model_orchestrator.create_tuning_pipeline()
  pipe.fit(dataset.X_train, dataset.y_train)
  y_pred = pipe.predict_proba(dataset.X_test)[:,1]
  auc = roc_auc_score(dataset.y_test, y_pred)
  return auc

study = create_study(study_name='optimization', direction='maximize', sampler = samplers.TPESampler())
study.optimize(lambda trial: objective(trial), n_trials=14)

[I 2024-03-27 08:10:33,348] A new study created in memory with name: optimization


 2024-03-27 08:10:33,365 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


 2024-03-27 08:10:38,220 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)
[I 2024-03-27 08:10:38,795] Trial 0 finished with value: 0.6866666929581499 and parameters: {'max_n_prebins': 86, 'learning_rate': 0.785746133297233, 'max_iter': 725, 'max_leaf_nodes': 33, 'max_depth': 20, 'l2_regularization': 0.4492001598442621, 'max_bins': 58}. Best is trial 0 with value: 0.6866666929581499.


...

c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)
[I 2024-03-27 08:12:57,542] Trial 12 finished with value: 0.6710705801079764 and parameters: {'max_n_prebins': 96, 'learning_rate': 0.021781109217318885, 'max_iter': 199, 'max_leaf_nodes': 67, 'max_depth': 1, 'l2_regularization': 0.9998743578965674, 'max_bins': 91}. Best is trial 10 with value: 0.6968864840970873.


 2024-03-27 08:12:57,558 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


 2024-03-27 08:13:06,686 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)
[I 2024-03-27 08:13:07,658] Trial 13 finished with value: 0.6960048956622488 and parameters: {'max_n_prebins': 80, 'learning_rate': 0.1647508040723532, 'max_iter': 293, 'max_leaf_nodes': 74, 'max_depth': 39, 'l2_regularization': 0.832648209403914, 'max_bins': 112}. Best is trial 10 with value: 0.6968864840970873.


In [18]:
for i, j in zip(study.best_params.keys(), study.best_params.values()):
    print(f"{i}: {j}")

max_n_prebins: 98
learning_rate: 0.03750655679690962
max_iter: 128
max_leaf_nodes: 78
max_depth: 37
l2_regularization: 0.9727372410566262
max_bins: 250


In [13]:
study.best_params.keys()

dict_keys(['max_n_prebins', 'learning_rate', 'max_iter', 'max_leaf_nodes', 'max_depth', 'l2_regularization', 'max_bins'])